# LangChain Tool, Tool Calling, Agent 튜토리얼

이 노트북은 LangChain의 핵심 개념인 **Tool**, **Tool Calling**, **Agent**를 단계별로 학습할 수 있도록 구성되었습니다.

## 📚 학습 목차

1. **Tool 기초** - 첫 번째 Tool 만들기
2. **Tool Calling** - LLM이 Tool을 호출하는 방법
3. **Agent** - 복잡한 문제를 단계별로 해결하기

---

## 🎯 학습 목표

이 노트북을 완료하면:
- Tool을 직접 만들 수 있습니다
- LLM이 Tool을 자동으로 사용하도록 설정할 수 있습니다  
- 복잡한 업무를 처리하는 Agent를 개발할 수 있습니다

---

## ⚙️ 환경 설정

먼저 필요한 패키지를 설치하고 API 키를 설정해야 합니다.


In [6]:
# 필요한 패키지 설치 (주석 해제해서 실행)
# !pip install langchain langchain-openai python-dotenv

# 패키지 import
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Annotated
import random
from datetime import datetime

# API 키 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 패키지 로드 완료!")
print("⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!")
print("   (.env 파일에 OPENAI_API_KEY=your_key_here)")


✅ 패키지 로드 완료!
⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!
   (.env 파일에 OPENAI_API_KEY=your_key_here)


# 1️⃣ Tool 기초

## Tool이란?
**Tool**은 LLM(AI 모델)이 특정 작업을 수행할 수 있도록 해주는 함수입니다.

예를 들어:
- 🧮 계산기 역할을 하는 Tool
- 🔍 웹 검색을 하는 Tool  
- 📁 파일을 읽는 Tool
- 💾 데이터베이스를 조회하는 Tool

LLM은 텍스트만 생성할 수 있지만, Tool을 통해 실제 작업을 수행할 수 있게 됩니다!

## 첫 번째 Tool 만들기


In [ ]:
# 가장 기본적인 Tool 만들기
@tool
def simple_calculator(
    a: Annotated[int, "첫 번째 숫자"], 
    b: Annotated[int, "두 번째 숫자"]
) -> int:
    """두 숫자를 더하는 간단한 계산기입니다."""
    result = a + b
    print(f"계산 결과: {a} + {b} = {result}")
    return result

# Tool의 정보 확인
print("=== Tool 정보 ===")
print(f"Tool 이름: {simple_calculator.name}")
print(f"Tool 설명: {simple_calculator.description}")
print(f"Tool 파라미터: {simple_calculator.args}")
print()

# Tool 직접 실행해보기
print("=== Tool 실행 ===")
result = simple_calculator.invoke({"a": 5, "b": 3}) ## 사람이 직접 Tool 호출 -> Tool의 기능 실행
print(f"직접 실행 결과: {result}")


## 더 복잡한 Tool들 만들기

이제 좀 더 실용적인 Tool들을 만들어봅시다.


In [ ]:
# 텍스트 분석 Tool
@tool
def text_analyzer(text: Annotated[str, "분석할 텍스트"]) -> dict:
    """텍스트를 분석해서 다양한 정보를 반환합니다."""
    
    words = text.split()
    word_count = len(words)
    char_count = len(text)
    char_count_no_space = len(text.replace(" ", ""))
    longest_word = max(words, key=len) if words else ""
    
    result = {
        "단어_개수": word_count,
        "문자_개수": char_count,
        "공백_제외_문자_개수": char_count_no_space,
        "가장_긴_단어": longest_word,
        "가장_긴_단어_길이": len(longest_word)
    }
    
    return result

# 가짜 날씨 정보 Tool
@tool
def weather_info(city: Annotated[str, "도시 이름"]) -> str:
    """가짜 날씨 정보를 제공합니다 (실제 API 연결 없음)."""
    
    temperatures = [15, 18, 22, 25, 28, 30, 12, 8]
    weather_conditions = ["맑음", "흐림", "비", "눈", "안개"]
    
    temp = random.choice(temperatures)
    condition = random.choice(weather_conditions)
    
    return f"{city}의 현재 날씨: {condition}, 기온: {temp}°C"

# Tool들 테스트
print("=== 텍스트 분석 Tool 테스트 ===")
sample_text = "안녕하세요! LangChain Tool을 배우고 있습니다. 정말 재미있어요!"
analysis_result = text_analyzer.invoke({"text": sample_text})
print(f"분석 결과: {analysis_result}")

print("\n=== 날씨 정보 Tool 테스트 ===")
weather_result = weather_info.invoke({"city": "서울"})
print(f"날씨 정보: {weather_result}")

# Tool 목록 관리
my_tools = [simple_calculator, text_analyzer, weather_info]
print(f"\n=== 사용 가능한 Tool 목록 ===")
for i, tool in enumerate(my_tools, 1):
    print(f"{i}. {tool.name}: {tool.description}")


## 🎯 Tool 핵심 포인트

- ✅ `@tool` 데코레이터로 일반 함수를 Tool로 변환
- ✅ `Annotated`로 파라미터 설명 추가
- ✅ docstring으로 Tool 기능 설명
- ✅ `tool.invoke()`로 직접 실행 가능
- ✅ 여러 Tool을 리스트로 관리

---

# 2️⃣ Tool Calling

## Tool Calling이란?
**Tool Calling**은 LLM이 주어진 문제를 해결하기 위해 적절한 Tool을 선택하고 실행하는 과정입니다.

### 과정:
1. 🗣️ 사용자가 질문을 합니다
2. 🧠 LLM이 질문을 분석해서 어떤 Tool이 필요한지 판단
3. 🔧 LLM이 Tool을 호출하고 결과를 받음
4. 💬 LLM이 결과를 바탕으로 최종 답변 생성

이렇게 하면 LLM이 계산, 검색, 데이터 처리 등을 할 수 있게 됩니다!


In [7]:
# 🔄 노트북 실행 시 주의사항: 이전 Tool들과 이름이 겹치지 않도록 주의!

# Tool Calling용 Tool들 준비하기 (새로운 이름으로)
@tool
def math_calculator(
    operation: Annotated[str, "연산 종류: add, subtract, multiply, divide"], 
    a: Annotated[float, "첫 번째 숫자"], 
    b: Annotated[float, "두 번째 숫자"]
) -> float:
    """수학 계산을 수행하는 계산기입니다."""
    
    if operation == "add":
        result = a + b
        print(f"계산: {a} + {b} = {result}")
    elif operation == "subtract":
        result = a - b
        print(f"계산: {a} - {b} = {result}")
    elif operation == "multiply":
        result = a * b
        print(f"계산: {a} × {b} = {result}")
    elif operation == "divide":
        if b == 0:
            return "0으로 나눌 수 없습니다!"
        result = a / b
        print(f"계산: {a} ÷ {b} = {result}")
    else:
        return f"지원하지 않는 연산입니다: {operation}"
    
    return result

@tool
def user_database(name: Annotated[str, "사용자 이름"]) -> dict:
    """사용자 정보를 조회하는 가짜 데이터베이스입니다."""
    
    # 가짜 사용자 데이터베이스
    users_db = {
        "김철수": {"나이": 25, "직업": "개발자", "취미": "독서"},
        "이영희": {"나이": 30, "직업": "디자이너", "취미": "그림그리기"},
        "박민수": {"나이": 28, "직업": "데이터분석가", "취미": "영화감상"},
    }
    
    if name in users_db:
        result = users_db[name]
        print(f"{name}님의 정보를 찾았습니다: {result}")
        return result
    else:
        return {"오류": f"{name}님의 정보를 찾을 수 없습니다."}

@tool  
def time_checker() -> str:
    """현재 시간을 반환합니다."""
    now = datetime.now()
    time_str = now.strftime("%Y년 %m월 %d일 %H시 %M분")
    print(f"현재 시간: {time_str}")
    return time_str

print("✅ Tool Calling용 Tool들이 준비되었습니다!")
print("- math_calculator: 사칙연산")
print("- user_database: 사용자 정보 조회") 
print("- time_checker: 현재 시간")
print()
print("💡 중요: 노트북에서는 Tool 이름이 중복되지 않도록 주의하세요!")


✅ Tool Calling용 Tool들이 준비되었습니다!
- math_calculator: 사칙연산
- user_database: 사용자 정보 조회
- time_checker: 현재 시간

💡 중요: 노트북에서는 Tool 이름이 중복되지 않도록 주의하세요!


## LLM에 Tool 연결하기

이제 LLM이 이 Tool들을 사용할 수 있도록 연결해봅시다!


In [8]:
# LLM 모델 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # 일관된 결과를 위해 0으로 설정
)

# Tool 목록 (새로운 이름으로 업데이트)
tools_for_calling = [math_calculator, user_database, time_checker]

# LLM에 Tool을 사용할 수 있도록 연결 ⭐ 이게 핵심!
llm_with_tools = llm.bind_tools(tools_for_calling)

print("🔗 LLM에 Tool들이 연결되었습니다!")
print("이제 LLM이 필요에 따라 Tool을 자동으로 사용할 수 있습니다!")


🔗 LLM에 Tool들이 연결되었습니다!
이제 LLM이 필요에 따라 Tool을 자동으로 사용할 수 있습니다!


## Tool Calling 실습

이제 LLM이 실제로 Tool을 호출하는지 확인해봅시다!


In [9]:
# 예시 1: 계산 요청
print("🔢 예시 1: 계산 요청")
user_question = "15에 23을 곱한 다음, 결과에서 47을 빼주세요"
print(f"사용자 질문: {user_question}")

# LLM에게 질문 전달
messages = [
    SystemMessage(content="당신은 도움이 되는 AI 어시스턴트입니다. 주어진 도구들을 사용해서 사용자의 질문에 답해주세요."),
    HumanMessage(content=user_question)
]

# LLM이 응답 생성 (Tool 호출 포함)
response = llm_with_tools.invoke(messages)

print(f"\nLLM 응답 내용: {response.content}")

# Tool 호출이 있는지 확인
if response.tool_calls:
    print(f"\n🔧 LLM이 호출한 Tool 개수: {len(response.tool_calls)}")
    
    for i, tool_call in enumerate(response.tool_calls, 1):
        print(f"\nTool 호출 {i}:")
        print(f"  - Tool 이름: {tool_call['name']}")
        print(f"  - 파라미터: {tool_call['args']}")
        
        # 실제로 Tool 실행
        for tool in tools_for_calling:
            if tool.name == tool_call['name']:
                result = tool.invoke(tool_call['args'])
                print(f"  - 실행 결과: {result}")
                break
else:
    print("Tool을 호출하지 않았습니다.")


🔢 예시 1: 계산 요청
사용자 질문: 15에 23을 곱한 다음, 결과에서 47을 빼주세요

LLM 응답 내용: 

🔧 LLM이 호출한 Tool 개수: 1

Tool 호출 1:
  - Tool 이름: math_calculator
  - 파라미터: {'operation': 'multiply', 'a': 15, 'b': 23}
계산: 15.0 × 23.0 = 345.0
  - 실행 결과: 345.0


In [ ]:
# 예시 2: 사용자 정보 조회
print("👤 예시 2: 사용자 정보 조회")
user_question = "김철수의 정보를 알려주세요"
print(f"사용자 질문: {user_question}")

messages = [
    SystemMessage(content="당신은 도움이 되는 AI 어시스턴트입니다. 주어진 도구들을 사용해서 사용자의 질문에 답해주세요."),
    HumanMessage(content=user_question)
]

response = llm_with_tools.invoke(messages)
print(f"응답 내용: {response.content}")

if response.tool_calls:
    print(f"\n🔧 LLM이 호출한 Tool:")
    for tool_call in response.tool_calls:
        print(f"  - Tool: {tool_call['name']}")
        print(f"  - 파라미터: {tool_call['args']}")
        
        # Tool 실행
        for tool in tools_for_calling:
            if tool.name == tool_call['name']:
                result = tool.invoke(tool_call['args'])
                print(f"  - 결과: {result}")
                break


## 🎯 Tool Calling 작동 원리

1. **🧠 LLM이 사용자 질문을 분석**
   - "15에 23을 곱한 다음, 47을 빼주세요"
   - → 이건 수학 계산이니까 calculator tool이 필요하겠네!

2. **🔧 LLM이 Tool 호출 계획 수립**
   - 첫 번째: calculator(operation="multiply", a=15, b=23)
   - 두 번째: calculator(operation="subtract", a=결과, b=47)

3. **⚡ Tool 실행**
   - 15 × 23 = 345
   - 345 - 47 = 298

4. **💬 최종 답변 생성**
   - "계산 결과는 298입니다."

---

# 3️⃣ Agent

## Agent란?
**Agent**는 복잡한 문제를 해결하기 위해 여러 Tool을 연속적으로 사용하는 더 똑똑한 시스템입니다.

### Tool Calling vs Agent:
- **Tool Calling**: LLM이 한 번에 Tool을 호출하고 끝
- **Agent**: 문제를 단계별로 나누어 여러 Tool을 순차적으로 사용

### Agent의 특징:
1. 🧠 **생각하기 (Reasoning)**: 문제를 분석하고 계획 수립
2. 🔧 **행동하기 (Action)**: 적절한 Tool 선택하고 실행
3. 👀 **관찰하기 (Observation)**: Tool 결과 확인
4. 🔄 **반복하기**: 목표 달성까지 2-3 과정 반복


In [10]:
# Agent용 Tool들 정의 (쇼핑 도우미 예시)
@tool
def search_product(query: Annotated[str, "검색할 상품명"]) -> list:
    """온라인 쇼핑몰에서 상품을 검색합니다."""
    
    # 가짜 상품 데이터베이스
    products = [
        {"이름": "무선 이어폰", "가격": 89000, "평점": 4.5, "재고": 15},
        {"이름": "블루투스 이어폰", "가격": 65000, "평점": 4.2, "재고": 8},
        {"이름": "노이즈캔슬링 헤드폰", "가격": 159000, "평점": 4.8, "재고": 3},
        {"이름": "게이밍 헤드셋", "가격": 95000, "평점": 4.3, "재고": 12},
    ]
    
    # 검색어와 관련된 상품 찾기
    results = []
    for product in products:
        if query.lower() in product["이름"].lower():
            results.append(product)
    
    print(f"'{query}' 검색 결과: {len(results)}개 상품 발견")
    return results

@tool
def check_inventory(product_name: Annotated[str, "상품명"]) -> dict:
    """특정 상품의 재고를 확인합니다."""
    
    inventory = {
        "무선 이어폰": {"재고": 15, "입고예정": "2024-01-15"},
        "블루투스 이어폰": {"재고": 8, "입고예정": "2024-01-10"},
        "노이즈캔슬링 헤드폰": {"재고": 3, "입고예정": "2024-01-20"},
        "게이밍 헤드셋": {"재고": 12, "입고예정": "2024-01-12"},
    }
    
    result = inventory.get(product_name, {"재고": 0, "입고예정": "미정"})
    print(f"'{product_name}' 재고 확인: {result}")
    return result

@tool
def calculate_shipping_cost(product_price: Annotated[float, "상품 가격"], quantity: Annotated[int, "수량"]) -> dict:
    """배송비를 계산합니다."""
    
    total_price = product_price * quantity
    
    if total_price >= 50000:
        shipping_cost = 0
        shipping_type = "무료배송"
    else:
        shipping_cost = 3000
        shipping_type = "일반배송"
    
    result = {
        "상품금액": total_price,
        "배송비": shipping_cost,
        "총금액": total_price + shipping_cost,
        "배송타입": shipping_type
    }
    
    print(f"배송비 계산 결과: {result}")
    return result

@tool
def add_to_cart(product_name: Annotated[str, "상품명"], quantity: Annotated[int, "수량"]) -> str:
    """상품을 장바구니에 추가합니다."""
    
    cart_id = random.randint(10000, 99999)
    message = f"'{product_name}' {quantity}개가 장바구니(ID: {cart_id})에 추가되었습니다."
    print(message)
    return message

print("🛒 쇼핑 Agent용 Tool들이 준비되었습니다!")
print("- search_product: 상품 검색")
print("- check_inventory: 재고 확인")
print("- calculate_shipping_cost: 배송비 계산")
print("- add_to_cart: 장바구니 추가")


ValidationError: 1 validation error for math_calculator
  Input should be a valid dictionary or instance of math_calculator [type=model_type, input_value=<function search_product at 0x107e06520>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

## Agent 생성하기

이제 여러 Tool을 연속적으로 사용하는 똑똑한 Agent를 만들어봅시다!


In [ ]:
# 쇼핑 도우미 Agent 생성
def create_shopping_agent():
    """쇼핑 도우미 Agent를 생성합니다."""
    
    # LLM 설정
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
    )
    
    # Agent용 Tool 목록
    agent_tools = [search_product, check_inventory, calculate_shipping_cost, add_to_cart]
    
    # Agent가 사용할 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 친절한 온라인 쇼핑 도우미 Agent입니다.
고객의 쇼핑을 도와주는 것이 목표입니다.

다음 도구들을 사용할 수 있습니다:
- search_product: 상품 검색
- check_inventory: 재고 확인  
- calculate_shipping_cost: 배송비 계산
- add_to_cart: 장바구니 추가

고객의 요청을 단계별로 처리해주세요:
1. 먼저 상품을 검색합니다
2. 재고를 확인합니다
3. 필요하면 배송비를 계산합니다
4. 고객이 원하면 장바구니에 추가합니다

항상 친절하고 도움이 되는 답변을 해주세요.
        """),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])
    
    # Agent 생성
    agent = create_tool_calling_agent(llm, agent_tools, prompt)
    
    # AgentExecutor 생성 (Agent를 실행하는 엔진)
    agent_executor = AgentExecutor(
        agent=agent,
        tools=agent_tools,
        verbose=True,  # Agent의 사고 과정을 보여줌
        max_iterations=10,  # 최대 반복 횟수
        handle_parsing_errors=True,  # 오류 처리
    )
    
    return agent_executor

# Agent 생성
shopping_agent = create_shopping_agent()
print("🤖 쇼핑 도우미 Agent가 준비되었습니다!")
print("복잡한 쇼핑 요청을 단계별로 처리할 수 있습니다.")


## Agent 실습

복잡한 쇼핑 요청을 Agent가 어떻게 단계별로 처리하는지 확인해봅시다!


In [ ]:
# 시나리오 1: 간단한 상품 검색
print("🛒 시나리오 1: 간단한 상품 검색")
print("="*50)

query1 = "이어폰을 찾고 있어요. 어떤 제품들이 있나요?"
print(f"고객 요청: {query1}")
print()

try:
    result1 = shopping_agent.invoke({"input": query1})
    print(f"\n🤖 Agent 최종 답변:")
    print(result1['output'])
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("API 키가 설정되어 있는지 확인해주세요!")


In [ ]:
# 시나리오 2: 복합적인 쇼핑 요청 (여러 Tool 사용)
print("\n" + "="*70)
print("🛒 시나리오 2: 복합적인 쇼핑 요청")
print("="*70)

query2 = "게이밍 헤드셋을 3개 주문하려고 합니다. 재고 확인하고, 배송비 포함 총액 계산해서 장바구니에 넣어주세요."
print(f"고객 요청: {query2}")
print()

try:
    result2 = shopping_agent.invoke({"input": query2})
    print(f"\n🤖 Agent 최종 답변:")
    print(result2['output'])
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("API 키가 설정되어 있는지 확인해주세요!")


# 🎓 최종 정리

## 배운 내용 요약

### 1️⃣ **Tool (도구)**
- `@tool` 데코레이터로 함수를 Tool로 변환
- LLM이 사용할 수 있는 기능 단위
- 예: 계산기, 검색기, 데이터 분석기

### 2️⃣ **Tool Calling (도구 호출)**
- LLM이 필요에 따라 Tool을 선택하고 실행
- `bind_tools()`로 LLM에 Tool 연결
- 단발성 작업에 적합

### 3️⃣ **Agent (에이전트)**
- 복잡한 문제를 단계별로 해결
- 여러 Tool을 연속적으로 사용
- **ReAct 패턴**: Reasoning → Action → Observation
- `AgentExecutor`로 실행

## 🔄 Agent의 사고 과정 (ReAct 패턴)

1. **Reasoning (추론)**: "고객이 이어폰을 원하니까 먼저 검색해야겠다"
2. **Action (행동)**: `search_product("이어폰")` 실행
3. **Observation (관찰)**: 검색 결과 확인
4. **Reasoning (추론)**: "재고도 확인해야겠다"
5. **Action (행동)**: `check_inventory("무선 이어폰")` 실행
... 반복 ...

## 🚀 다음 단계
- 더 복잡한 Agent 설계
- 외부 API 연동 Tool 개발  
- 메모리 기능 추가
- 멀티 Agent 시스템 구축

---

**축하합니다! Tool, Tool Calling, Agent의 핵심을 모두 이해했습니다!** 🎉

이제 실무에서 바로 활용할 수 있는 AI Agent를 개발할 수 있습니다!
